## Install kglab and Pytorch Geometric

In [1]:
# !pip install --upgrade pip

In [2]:

# version = "1.12.1" # original: 1.8.0
version = "1.8.0"
# %pip install torch-scatter -f f"https://pytorch-geometric.com/whl/torch-{version}+cpu.html" -qqq
# %pip install torch-sparse -f f"https://pytorch-geometric.com/whl/torch-{version}+cpu.html" -qqq
# %pip install torch-cluster -f f"https://pytorch-geometric.com/whl/torch-{version}+cpu.html" -qqq
# %pip install torch-spline-conv -f f"https://pytorch-geometric.com/whl/torch-{version}+cpu.html" -qqq

# # Consider install 1.12.1 version instead of 1.8.0

# %pip install torch-geometric
# %pip install torch==1.8.0 -qqq
# # %pip install torch==1.12.1 -qqq

# %pip install kglab==0.5.1 -qqq

# # %pip install pytorch_lightning -qq

In [8]:
!pip install --no-index torch-scatter -f "https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html"

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html


In [9]:
!pip install --no-index torch-sparse -f "https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html"

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html


In [10]:
!pip install --no-index torch-cluster -f "https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html"

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html


In [11]:
!pip install --no-index torch-spline-conv -f "https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html"

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html


In [12]:
!pip install torch-geometric

In [15]:
!pip install torch==1.8.0

In [9]:
!pip install kglab==0.5.1

In [10]:
!pip install pytorch_lightning

  Using cached torch-1.12.1-cp37-cp37m-win_amd64.whl (161.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


In [4]:
import kglab

NAMESPACES = {
    "wtm":  "http://purl.org/heals/food/",
    "ind":  "http://purl.org/heals/ingredient/",
    "recipe":  "https://www.food.com/recipe/",
    }

kg = kglab.KnowledgeGraph(namespaces = NAMESPACES)

# _ = kg.load_rdf("data/recipe_lg.ttl")
# _ = kg.load_rdf("/content/drive/MyDrive/CIC/RDFs/recipes.ttl")
_ = kg.load_rdf("recipes.ttl")


## Algunas métricas

In [5]:
measure = kglab.Measure()
measure.measure_graph(kg)

f"Nodes: {measure.get_node_count()} ; Edges: {measure.get_edge_count()}"

'Nodes: 286 ; Edges: 1891'

In [18]:
measure.p_gen.get_tally() # tallies the counts of predicates

,count
http://purl.org/heals/food/hasIngredient,1078
http://www.w3.org/1999/02/22-rdf-syntax-ns#type,286
http://www.w3.org/2004/02/skos/core#definition,249
http://purl.org/heals/food/hasCookTime,240
http://www.w3.org/2004/02/skos/core#prefLabel,9
http://purl.org/dc/terms/identifier,7
http://www.w3.org/2004/02/skos/core#altLabel,5
http://www.w3.org/2004/02/skos/core#narrower,5
http://www.w3.org/2004/02/skos/core#closeMatch,3
http://www.w3.org/2000/01/rdf-schema#range,2


In [14]:
measure.s_gen.get_tally()[:10] # tallies the counts of subjects

,count
https://www.food.com/recipe/103964,11
https://www.food.com/recipe/220361,10
https://www.food.com/recipe/137158,10
https://www.food.com/recipe/123656,10
https://www.food.com/recipe/111008,10
https://www.food.com/recipe/151617,10
https://www.food.com/recipe/268209,10
http://example.org/#Pancake,10
https://www.food.com/recipe/135405,10
https://www.food.com/recipe/430777,10


In [15]:
measure.o_gen.get_tally()[:10] # tallies the counts of objects

,count
http://purl.org/heals/food/Recipe,240
http://purl.org/heals/ingredient/AllPurposeFlour,190
http://purl.org/heals/ingredient/ChickenEgg,170
http://purl.org/heals/ingredient/Salt,158
http://purl.org/heals/ingredient/Butter,147
http://purl.org/heals/ingredient/CowMilk,132
http://purl.org/heals/ingredient/WhiteSugar,99
http://purl.org/heals/ingredient/Water,70
http://purl.org/heals/ingredient/VanillaExtract,54
http://example.org/#Noodle,21


In [16]:
measure.l_gen.get_tally()[:10] # tallies the counts of literals

,count
PT30M,29
PT20M,21
PT25M,20
PT35M,18
PT40M,18
PT10M,12
PT15M,11
PT1H,10
PT5M,8
PT55M,7


## KG --> Pytorch Tensor

In [6]:
sg = kglab.SubgraphTensor(kg)

In [8]:
def to_edge_list(g, sg, excludes):
    def exclude(rel):
        return sg.n3fy(rel) in excludes

    relations = sorted(set(g.predicates()))
    subjects = set(g.subjects())
    objects = set(g.objects())
    nodes = list(subjects.union(objects))

    relations_dict = {rel: i for i, rel in enumerate(list(relations)) if not exclude(rel)}

    # this offset enables consecutive indices in our final vector
    offset = len(relations_dict.keys())

    nodes_dict = {node: i+offset for i, node in enumerate(nodes)}


    edge_list = []

    for s, p, o in g.triples((None, None, None)):
        if p in relations_dict.keys(): # this means is not excluded
            src, dst, rel = nodes_dict[s], nodes_dict[o], relations_dict[p]
            edge_list.append([src, dst, 2 * rel])
            edge_list.append([dst, src, 2 * rel + 1])

    # turn into str keys and concat
    node_vector = [sg.n3fy(node) for node in relations_dict.keys()] + [sg.n3fy(node) for node in nodes_dict.keys()]
    return edge_list, node_vector

In [9]:
edge_list, node_vector = to_edge_list(kg.rdf_graph(), sg, excludes=['skos:description', 'skos:prefLabel'])

In [22]:
len(edge_list) , edge_list[0:5]

(3764,
 [[156, 214, 12],
  [214, 156, 13],
  [442, 448, 10],
  [448, 442, 11],
  [354, 214, 12]])

In [21]:
node_vector[:10]

['nom:madeFrom',
 'nom:usesProcess',
 'dct:identifier',
 'dct:publisher',
 'wtm:hasCookTime',
 'wtm:hasIngredient',
 'rdf:type',
 'rdfs:domain',
 'rdfs:range',
 'skos:altLabel']

## Diccionario de datos para codificar y decodificar datos

In [10]:
sg = kglab.Subgraph(kg=kg, preload=node_vector)

In [11]:
import torch
from torch_geometric.data import Data

tensor = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
edge_index, edge_type = tensor[:2], tensor[2]
data = Data(edge_index=edge_index)
data.edge_type = edge_type

In [ ]:
(data.edge_index.shape, data.edge_type.shape, data.edge_type.max())


## Creando training set con Rubrix
### -> Uso de un método supervisado "sencillo" (snorkel?)
### -> rb.log para registrar datos sin clasificación
### -> Uso de webapp para asociar etiquetas definidas por nosotros (aprendizaje semisupervisado ?)
### -> rb.load() regresa un pd.Dataframe para luego hacer uso de la función sklearn.train_test_split()
### -> Después del entrenamiento se puede clasificar los datos sin etiqueta.

In [12]:
LABELS = ['Bitter', 'Meaty', 'Piquant', 'Salty', 'Sour', 'Sweet']


# Rubrix setup

In [53]:
# !pip install rubrix==0.1.0
# !pip install rubrix
!pip install rubrix==0.6.2
# !pip install "rubrix[server]"==0.1.0
!pip install loguru

     ---------------------------------------- 1.1/1.1 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: rubrix
    Found existing installation: rubrix 0.1.0
    Uninstalling rubrix-0.1.0:
      Successfully uninstalled rubrix-0.1.0


In [54]:
import rubrix as rb


# Preparando un "raw" dataset de recetas e ingredientes

In [55]:
import pandas as pd
sparql = """
    SELECT distinct *
    WHERE {
        ?uri a wtm:Recipe .
        ?uri a ?type .
        ?uri skos:definition ?definition .
        ?uri wtm:hasIngredient ?ingredient
    }
"""
df = kg.query_as_df(sparql=sparql)

# We group the ingredients into one column containing lists:
recipes_df = df.groupby(['uri', 'definition', 'type'])['ingredient'].apply(list).reset_index(name='ingredients') ; recipes_df

sparql_ingredients = """
    SELECT distinct *
    WHERE {
        ?uri a wtm:Ingredient .
        ?uri a ?type .
        OPTIONAL { ?uri skos:prefLabel ?definition }
    }
"""

df = kg.query_as_df(sparql=sparql_ingredients)
df['ingredients'] = None

ing_recipes_df = pd.concat([recipes_df, df]).reset_index(drop=True)

ing_recipes_df.fillna('', inplace=True) ; ing_recipes_df


,uri,definition,type,ingredients
0,recipe:100230,astoria frosting,wtm:Recipe,"[ind:AllPurposeFlour, ind:Butter, ind:CowMilk,..."
1,recipe:101876,wheat thins,wtm:Recipe,"[ind:Butter, ind:Salt, ind:VanillaExtract, ind..."
2,recipe:103073,simple popovers,wtm:Recipe,"[ind:AllPurposeFlour, ind:Butter, ind:ChickenE..."
3,recipe:103964,spatzle little german noodle dumplings,nom:Noodle,"[ind:AllPurposeFlour, ind:BlackPepper, ind:But..."
4,recipe:103964,spatzle little german noodle dumplings,wtm:Recipe,"[ind:AllPurposeFlour, ind:BlackPepper, ind:But..."
...,...,...,...,...
272,recipe:93990,egg roll wrappers,wtm:Recipe,"[ind:AllPurposeFlour, ind:ChickenEgg, ind:Salt..."
273,recipe:94616,milk frosting,wtm:Recipe,"[ind:Butter, ind:CowMilk, ind:VanillaExtract, ..."
274,recipe:95116,the easiest shortcrust pastry,wtm:Recipe,"[ind:AllPurposeFlour, ind:Butter, ind:WhiteSugar]"
275,recipe:96087,fresh made manicotti or cannelloni crepes shells,wtm:Recipe,"[ind:AllPurposeFlour, ind:ChickenEgg, ind:CowM..."


# Logging into Rubrix

In [56]:
import rubrix as rb

records = []
for i, r in ing_recipes_df.iterrows():
    item = rb.TextClassificationRecord(
            inputs={
                "id":r.uri,
                "definition": r.definition,
                "ingredients": str(r.ingredients),
                "type": r.type
            }, # log node fields
            prediction=[(label, 0.0) for label in LABELS], # log "dummy" predictions for aiding annotation
            metadata={'ingredients': [ing.replace('ind:','') for ing in r.ingredients], "type": r.type}, # metadata filters for quick exploration and annotation
            prediction_agent="kglab_tutorial", # who's performing/logging the prediction
            multi_label=True
        )
    records.append(item)


In [57]:
len(records)


277

In [58]:
rb.log(records=records, name="kg_classification_tutorial")


100%|██████████| 277/277 [00:09<00:00, 29.97it/s]

277 records logged to http://localhost:6900/datasets/rubrix/kg_classification_tutorial


BulkResponse(dataset='kg_classification_tutorial', processed=277, failed=0)

In [59]:
import rubrix as rb
rb.snapshots(name="kg_classification_tutorial")


AttributeError: module rubrix has no attribute snapshots

In [ ]:
dir(rb)

In [69]:
# df = rb.load(name="kg_classification_tutorial", snapshot='1620136587.907149') ; df.head()
df = rb.load(name="kg_classification_tutorial", as_pandas=True) ; df.head()


ValueError: ('The argument `as_pandas` is deprecated and will be removed in a future version. Please adapt your code accordingly. ', "If you want a pandas DataFrame do `rb.load('my_dataset').to_pandas()`.")

In [89]:
df.head(5)

,text,inputs,prediction,prediction_agent,annotation,annotation_agent,multi_label,explanation,id,metadata,status,event_timestamp,metrics,search_keywords
0,None,"{'id': 'recipe:437729', 'definition': 'super e...","[(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...",kglab_tutorial,None,None,True,None,03437962-13a7-4e98-aa03-b762e08c2d44,"{'ingredients': ['AllPurposeFlour', 'Butter', ...",Default,None,{'text_length': 117},None
1,None,"{'id': 'recipe:221279', 'definition': 'paula d...","[(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...",kglab_tutorial,None,None,True,None,03496970-4751-41f9-8339-f761814d8f3f,"{'ingredients': ['AllPurposeFlour', 'ChickenEg...",Default,None,{'text_length': 115},None
2,None,"{'id': 'recipe:107499', 'definition': 'scandin...","[(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...",kglab_tutorial,None,None,True,None,051d8f98-c607-469f-97a7-3e17dd487125,"{'ingredients': ['AllPurposeFlour', 'Butter', ...",Default,None,{'text_length': 146},None
3,None,"{'id': 'recipe:382707', 'definition': 'cottage...","[(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...",kglab_tutorial,None,None,True,None,0554805e-c700-477a-8ed9-bf18b819f44b,"{'ingredients': ['AppleCiderVinegar', 'CowMilk...",Default,None,{'text_length': 92},None
4,None,"{'id': 'recipe:186307', 'definition': 'suzi s ...","[(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...",kglab_tutorial,None,None,True,None,0606c0a9-9b91-40ed-8801-09b8f1208dec,"{'ingredients': ['AllPurposeFlour', 'Butter', ...",Default,None,{'text_length': 145},None


In [83]:
df.iloc[0]

text                                                             None
inputs              {'id': 'recipe:437729', 'definition': 'super e...
prediction          [(Bitter, 0.0), (Meaty, 0.0), (Piquant, 0.0), ...
prediction_agent                                       kglab_tutorial
annotation                                                       None
annotation_agent                                                 None
multi_label                                                      True
explanation                                                      None
id                               03437962-13a7-4e98-aa03-b762e08c2d44
metadata            {'ingredients': ['AllPurposeFlour', 'Butter', ...
status                                                        Default
event_timestamp                                                  None
metrics                                          {'text_length': 117}
search_keywords                                                  None
Name: 0, dtype: obje

In [50]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df)
train_df.to_csv('data/train_recipes_new.csv')
test_df.to_csv('data/test_recipes_new.csv')

# Datasets train y test para Pytorch

In [106]:
import pandas as pd

train_df = pd.read_csv('data/train_recipes_1.csv') # use your own labelled datasets if you've created a snapshot
test_df = pd.read_csv('data/test_recipes_1.csv')

# # we make sure lists are parsed correctly
train_df.labels = train_df.labels.apply(eval)
test_df.labels = test_df.labels.apply(eval)
# train_df.labels = train_df.prediction.apply(eval)
# test_df.labels = test_df.prediction.apply(eval)

## Codificacion de etiquetas y diccionario apra revertir decodificar

In [107]:
label2id = {label:i for i,label in enumerate(LABELS)} ;
id2label = {i:l for l,i in label2id.items()} ; (id2label, label2id)

({0: 'Bitter', 1: 'Meaty', 2: 'Piquant', 3: 'Salty', 4: 'Sour', 5: 'Sweet'},
 {'Bitter': 0, 'Meaty': 1, 'Piquant': 2, 'Salty': 3, 'Sour': 4, 'Sweet': 5})

## Dataframe --> Arrays numéricos para índices y etiquetas

In [111]:
import numpy as np

def create_indices_labels(df):
    # turn our dense labels into a one-hot list
    def one_hot(label_ids):
        a = np.zeros(len(LABELS))
        a.put(label_ids, np.ones(len(label_ids)))
        return a

    indices, labels = [], []
    for uri, label in zip(df.uri.tolist(), df.labels.tolist()):
        indices.append(sg.transform(uri))
        labels.append(one_hot([label2id[label] for label in label]))
    # for id, label in zip(df.id.tolist(), df.labels.tolist()):
    #     indices.append(sg.transform(id))
    #     # labels.append(one_hot([label2id[label] for label in label]))
    #     labels.append(one_hot([label2id[(label[0], int(label[1]))] for label in label]))
    return indices, labels

In [112]:
train_indices, train_labels = create_indices_labels(train_df)
test_indices, test_labels = create_indices_labels(test_df)

train_idx = torch.tensor(train_indices, dtype=torch.long)
train_y = torch.tensor(train_labels, dtype=torch.float)

test_idx = torch.tensor(test_indices, dtype=torch.long)
test_y = torch.tensor(test_labels, dtype=torch.float) ; train_idx[:10], train_y

(tensor([618, 619, 620, 621, 622, 623, 624, 625, 626, 627]),
 tensor([[0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 1.],
         ...,
         [0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [1., 0., 1., 0., 0., 1.]]))

## Corroborando que se pueda recuperar URIs de manera correcta usando kglab.Subgraph

In [154]:
len(train_idx), len(train_y)

(441, 441)

In [114]:
# (train_df.loc[0], sg.inverse_transform(15380))


---
# Creación de subgrafo de recetas e ingredientes

### Creación de "semilla"
### Objetivo: Clasificar nodos

In [115]:
node_idx = torch.LongTensor([
    sg.transform(i) for i in ing_recipes_df.uri.values
])


In [116]:
node_idx.max(), node_idx.shape


(tensor(615), torch.Size([277]))

In [117]:
ing_recipes_df.iloc[1]


uri                                                recipe:101876
definition                                           wheat thins
type                                                  wtm:Recipe
ingredients    [ind:Butter, ind:Salt, ind:VanillaExtract, ind...
Name: 1, dtype: object

In [118]:
sg.inverse_transform(node_idx[1])


'recipe:101876'

In [119]:
node_idx[0:10]

tensor([324, 381, 181, 230, 230, 183, 448, 109,  60, 346])

# Clasificación Semi-supervisada de nodos con PyTorch Geometric
## Se usará una variante de las clásicas GNN --> Relational Graph Convolutional Network 
## >> MULTI-RELATIONAL KNOWLEDGE GRAPHS <<

### Creación de subgrafo tipo PyG

In [120]:
from torch_geometric.utils import k_hop_subgraph
# here we take all connected nodes with k hops
k = 1
node_idx, edge_index, mapping, edge_mask = k_hop_subgraph(
    node_idx,
    k,
    data.edge_index,
    relabel_nodes=False
)


In [121]:
node_idx.shape


torch.Size([553])

In [122]:
data.edge_index.shape


torch.Size([2, 3764])

#### Algunas métricas

In [123]:
data.edge_index = edge_index

data.num_nodes = data.edge_index.max().item() + 1

data.num_relations = data.edge_type.max().item() + 1

data.edge_type = data.edge_type[edge_mask]

data.num_classes = len(LABELS)

data.num_nodes, data.num_relations, data.num_classes

(616, 26, 6)

## Definición básica de una RGCN

In [124]:
from torch_geometric.nn import FastRGCNConv, RGCNConv
import torch.nn.functional as F

In [ ]:
RGCNConv?


In [125]:
class RGCN(torch.nn.Module):
    def __init__(self, num_nodes, num_relations, num_classes, out_channels=16, num_bases=30, dropout=0.0, layer_type=FastRGCNConv, ):

        super(RGCN, self).__init__()

        self.conv1 = layer_type(
            num_nodes,
            out_channels,
            num_relations,
            num_bases=num_bases
        )
        self.conv2 = layer_type(
            out_channels,
            num_classes,
            num_relations,
            num_bases=num_bases
        )
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, edge_index, edge_type):
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.dropout(x)
        x = self.conv2(x, edge_index, edge_type)
        return torch.sigmoid(x)

## Crea y visualiza el modelo

In [136]:
model = RGCN(
    num_nodes=data.num_nodes,
    num_relations=data.num_relations,
    num_classes=data.num_classes,
    out_channels=64,
    dropout=0.2,
    layer_type=RGCNConv
) ; model

RGCN(
  (conv1): RGCNConv(616, 64, num_relations=26)
  (conv2): RGCNConv(64, 6, num_relations=26)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
!pip install matplotlib

In [139]:
# code adapted from https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# from pytorch_lightning.metrics.utils import to_categorical

def visualize(h, color, labels):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    scatter = plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    legend = plt.legend(scatter.legend_elements()[0],labels, loc="upper right", title="Labels",) #*scatter.legend_elements()
    plt.show()

In [140]:
pred = model(edge_index, edge_type)


IndexError: The shape of the mask [3764] at index 0 does not match the shape of the indexed tensor [2, 3670] at index 1

In [133]:
visualize(pred[train_idx], color=to_categorical(train_y), labels=LABELS)


NameError: name 'pred' is not defined

In [ ]:
visualize(pred[test_idx], color=to_categorical(test_y), labels=LABELS)


## Entrena la red neuronal

In [162]:
device = torch.device('cpu') # ('cuda')
data = data.to(device)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters())
loss_module = torch.nn.BCELoss()

def train():
    print(len(data.edge_index[0]), len(data.edge_type))
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type)
    print(len(out))
    loss = loss_module(out[train_idx], train_y)
    loss.backward()
    optimizer.step()
    return loss.item()

def accuracy(predictions, y):
    predictions = np.round(predictions)
    return predictions.eq(y).to(torch.float).mean()

@torch.no_grad()
def test():
    model.eval()
    pred = model(data.edge_index, data.edge_type)
    train_acc = accuracy(pred[train_idx], train_y)
    test_acc = accuracy(pred[test_idx], test_y)
    return train_acc.item(), test_acc.item()

In [158]:
train_idx.unique

<bound method Tensor.unique of tensor([ 618,  619,  620,  621,  622,  623,  624,  625,  626,  627,  628,  629,
         630,  631,  632,  633,  634,  635,  636,  637,  638,  639,  640,  641,
         642,  643,  644,  645,  646,  647,  648,  649,  650,  651,  652,  653,
         654,  655,  656,  657,  658,  659,  660,  661,  662,  663,  140,  664,
         665,  666,  667,  668,  669,  670,  671,  672,  673,  674,  675,  676,
         677,  678,  679,  680,  681,  682,  683,  684,  685,  686,  687,  688,
         689,  690,  691,  692,  693,  694,  695,  696,  697,  698,  699,  700,
         701,  702,  703,  704,  705,  706,  707,  708,  709,  118,  710,  711,
         712,  713,  714,  715,  716,  717,  718,  719,  720,  721,  722,  723,
         724,  725,  726,  727,  728,  729,  730,  731,  732,  733,  734,  735,
         736,  737,  738,  739,  740,  741,  742,  743,  744,  745,  746,  747,
         748,  749,  750,  751,  752,  753,  754,  755,  756,  757,  758,  759,
         

In [163]:
for epoch in range(1, 50):
    loss = train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
          f'Test: {test_acc:.4f}')

3670 3670
616


IndexError: index 618 is out of bounds for dimension 0 with size 616

## Visualización del modelo

In [ ]:
pred = model(edge_index, edge_type)


In [ ]:
visualize(pred[train_idx], color=to_categorical(train_y), labels=LABELS)


In [ ]:
visualize(pred[test_idx], color=to_categorical(test_y), labels=LABELS)


# Uso y análisis de las predicciones mediante Rubrix

In [ ]:
pred = model(edge_index, edge_type) ; pred


In [ ]:
def find(tensor, values):
    return torch.nonzero(tensor[..., None] == values)

### Análisis de predicciones Vs test dataset

In [ ]:
test_idx = find(node_idx,test_idx)[:,0] ; len(test_idx)


In [ ]:
index = torch.zeros(node_idx.shape[0], dtype=bool)
index[test_idx] = True
idx = node_idx[index]

In [ ]:
uris = [sg.inverse_transform(i) for i in idx]
predicted_labels = [l for l in pred[idx]]

In [ ]:
predictions = list(zip(uris,predicted_labels)) ; predictions[0:2]

In [ ]:
import rubrix as rb

records = []
for uri,predicted_labels in predictions:
    ids = ing_recipes_df.index[ing_recipes_df.uri == uri]
    if len(ids) > 0:
        r = ing_recipes_df.iloc[ids]
        # get the gold labels from our test set
        gold_labels = test_df.iloc[test_df.index[test_df.uri == uri]].labels.values[0]

        item = rb.TextClassificationRecord(
                inputs={"id":r.uri.values[0], "definition": r.definition.values[0], "ingredients": str(r.ingredients.values[0]), "type": r.type.values[0]},
                prediction=[(id2label[i], score) for i,score in enumerate(predicted_labels)],
                annotation=gold_labels,
                metadata={'ingredients': r.ingredients.values[0], "type": r.type.values[0]},
                prediction_agent="node_classifier_v1",
                multi_label=True
        )
        records.append(item)

In [ ]:
rb.log(records, name="kg_classification_test_analysis")


### Analisis de predicciones ahora con datos no usados (reclasificación?)

In [ ]:
train_test_idx = find(node_idx,torch.cat((test_idx, train_idx)))[:,0] ; len(train_test_idx)


In [ ]:
# Ids que no están en el training dataset
index = torch.ones(node_idx.shape[0], dtype=bool)
index[train_test_idx] = False
idx = node_idx[index]

In [ ]:
# SubgraphTensor para des-transformar las URIs
uris = [sg.inverse_transform(i) for i in idx]
predicted_labels = [l for l in pred[idx]]

In [ ]:
predictions = list(zip(uris,predicted_labels)) ; predictions[0:2]


In [ ]:
import rubrix as rb

records = []
for uri,predicted_labels in predictions:
    ids = ing_recipes_df.index[ing_recipes_df.uri == uri]
    if len(ids) > 0:
        r = ing_recipes_df.iloc[ids]
        item = rb.TextClassificationRecord(
                inputs={"id":r.uri.values[0], "definition": r.definition.values[0], "ingredients": str(r.ingredients.values[0]), "type": r.type.values[0]},
                prediction=[(id2label[i], score) for i,score in enumerate(predicted_labels)],
                metadata={'ingredients': r.ingredients.values[0], "type": r.type.values[0]},
                prediction_agent="node_classifier_v1",
                multi_label=True
        )
        records.append(item)

In [ ]:
rb.log(records, name="kg_node_classification_unseen_nodes_v3")
